# Mergekit
- LLM을 병합하는 툴킷

## Merge Algorithm
1. SLERP (Spherical Linear Interpolation)
    - 두 벡터를 보간하기 위한 방법
    - 두개의 모델로 이용

2. TIES
    - 멀티 태스킹 가능

3. DARE
    - Pruning, Rescaling 방식이 존재함

4. Passthrough
    - 덩치가 큰 모델을 만드는 방법 (레이어를 합쳐서 수행)

## 각 Merge 방법의 yaml 파일

### TIES-Merging

```yaml
models:
  - model: mistralai/Mistral-7B-v0.1
    # no parameters necessary for base model
  - model: OpenPipe/mistral-ft-optimized-1218
    parameters:
      density: 0.5
      weight: 0.5
  - model: mlabonne/NeuralHermes-2.5-Mistral-7B
    parameters:
      density: 0.5
      weight: 0.3
merge_method: ties
base_model: mistralai/Mistral-7B-v0.1
parameters:
  normalize: true
dtype: float16
```

### SLERP

```yaml
slices:
  - sources:
      - model: OpenPipe/mistral-ft-optimized-1218
        layer_range: [0, 32]
      - model: mlabonne/NeuralHermes-2.5-Mistral-7B
        layer_range: [0, 32]
merge_method: slerp
base_model: OpenPipe/mistral-ft-optimized-1218
parameters:
  t:
    - filter: self_attn
      value: [0, 0.5, 0.3, 0.7, 1]
    - filter: mlp
      value: [1, 0.5, 0.7, 0.3, 0]
    - value: 0.5
dtype: bfloat16
```

### Passthrough

```yaml
slices:
  - sources:
    - model: OpenPipe/mistral-ft-optimized-1218
      layer_range: [0, 32]
  - sources:
    - model: mlabonne/NeuralHermes-2.5-Mistral-7B
      layer_range: [24, 32]
merge_method: passthrough
dtype: bfloat16
```

In [ ]:
!git clone https://github.com/cg123/mergekit.git
!cd mergekit && pip install -q -e .

In [ ]:
from huggingface_hub import login
login()

In [ ]:
import yaml

MODEL_NAME = "Marcoro14-7B-slerp_v2"
yaml_config = """
slices:
  - sources:
      - model: OpenPipe/mistral-ft-optimized-1218
        layer_range: [0, 32]
      - model: mlabonne/NeuralHermes-2.5-Mistral-7B
        layer_range: [0, 32]
merge_method: slerp
base_model: OpenPipe/mistral-ft-optimized-1218
parameters:
  t:
    - filter: self_attn
      value: [0, 0.5, 0.3, 0.7, 1]
    - filter: mlp
      value: [1, 0.5, 0.7, 0.3, 0]
    - value: 0.5
dtype: bfloat16
"""

with open('config.yaml', 'w', encoding="utf-8") as f:
    f.write(yaml_config)

In [ ]:
# Merge Models
!mergekit-yaml config.yaml merge --copy-tokenizer --allow-crimes --out-shard-size 1B --lazy-unpickle

In [ ]:
!pip install -qU huggingface_hub

from huggingface_hub import ModelCard, ModelCardData
from jinja2 import Template

usernmae = "wonik-hi"

template_text = """
---
license: apache-2.0
tags:
- merge
- mergekit
- lazymergekit
{%- for model in models %}
- {{ model }}
{%- endfor %}
---

# {{ model_name }}

{{ model_name }} is a merge of the following models using [mergekit](https://github.com/cg123/mergekit):

{%- for model in models %}
* [{{ model }}](https://huggingface.co/{{ model }})
{%- endfor %}

## 🧩 Configuration

```yaml
{{- yaml_config -}}
```
"""


# Create a Jinja template object
jinja_template = Template(template_text.strip())

# Get list of models from config
data = yaml.safe_load(yaml_config)
if "models" in data:
    models = [data["models"][i]["model"] for i in range(len(data["models"])) if "parameters" in data["models"][i]]
elif "parameters" in data:
    models = [data["slices"][0]["sources"][i]["model"] for i in range(len(data["slices"][0]["sources"]))]
elif "slices" in data:
    models = [data["slices"][i]["sources"][0]["model"] for i in range(len(data["slices"]))]
else:
    raise Exception("No models or slices found in yaml config")


# Fill the template
content = jinja_template.render(
    model_name=MODEL_NAME,
    models=models,
    yaml_config=yaml_config,
    username=username,
)

# Save the model card
card = ModelCard(content)
card.save('merge/README.md')

In [ ]:
f"{username}/{MODEL_NAME}"

In [ ]:
from google.coalb import userdata
from huggingface_hub import HfApi

username = "wonik-hi"

api = HfAPi(token=userdata.get("HF_API_KEY2"))

"""
api.create_repo(
    repo_id=f"{username}/MODEL_NAME}",
    repo_type="model"
)
"""
api.upload_folder(
    repo_id=f"{username}/{MODEL_NAME}"
)

### TIES Merge 방법

In [ ]:
import yaml

MODEL_NAME = "wonik-hi_merge_model"
yaml_config = """
models:
  - model: wonik-hi/Marcoro14-7B-slerp
    parameters:
      density: [1, 0.7, 0.1] # density gradient
      weight: 1.0
  - model: mlabonne/NeuralHermes-2.5-Mistral-7B
    parameters:
      density: 0.33
      weight:
        - filter: mlp
          value: 0.5
        - value: 0
merge_method: ties
base_model: mistralai/Mistral-7B-v0.1
parameters:
  normalize: true
  int8_mask: true
dtype: float16
name: gradient-slerp-ties
"""

# Save config as yaml file
with open('/content/conf.yml', 'w', encoding="utf-8") as f:
    f.write(yaml_config)

In [ ]:
# 변수 셋팅
OUTPUT_PATH = "./merged_v2" # 저장될 경로
LORA_MERGE_CACHE = "/tmp" # 로라 병합 캐시 저장 경로
CONFIG_YML = "/content/conf.yml" # config yml 파일 경로
COPY_TOKENIZER = True
LAZY_UNPICKLE = True # 실험적인 저용량 모델 로더 기능 활성화
LOW_CPU_MEMORY = True # 서버 성능 좋으면 True 설정
ALLOW_CRIMES = True

In [ ]:
!git clone https://github.com/cg123/mergekit.git

In [ ]:
!ls

In [ ]:
%cd mergekit

In [ ]:
# 깃에 셋팅된 패키지 설치
!pip install -e .

In [ ]:
# 패키지 임포트
import torch
import yaml

from mergekit.config import MergeConfiguration
from mergekit.merge import MergeOptions, run_merge

# yml 파일 설정값 불러오기
with open(CONFIG_YML, "r", encoding="utf-8") as fp:
    merge_config = MergeConfiguration.model_validate(yaml.safe_load(fp))


run_merge(
    merge_config,
    out_path=MergeOptions(
        lora_merge_cache=LORA_MERGE_CACHE,
        cuda=torch.cuda.is_available(), # GPU 세팅
        copy_tokenizer=COPY_TOKENIZER, # 토크나이저
        lazy_unpickle=LAZY_UNPICKLE,
        low_cpu_memory=LOW_CPU_MEMORY,
        allow_crimes=ALLOW_CRIMES,
    ),
)
print("Done")

In [ ]:
from google.colab import userdata
from huggingface_hub import HfApi

username = "wonik-hi"

api = HfApi(token=userdata.get("huggingface"))

"""
api.create_repo(
    repo_id=f"{username}/{MODEL_NAME}",
    repo_type="model"
)
"""

api.upload_folder(
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path="merged_v2",
)

## Model Evaluation
- 실행환경 : T4 정도의 GPU 성능 필요
- KoBEST : Korean Balanced Evaluation of Singificant Tasks
잘 구성된 벤치마크는 다양한 모델을 객관적이고 정밀하게 평가 -> 자연어 처리 분야의 발전을 촉진

- Ko LM Eval Harness : 한국어 공개 데이터셋으로 평가하는 언어 모델 점수
    - 크기가 크기 때문에 colab에서 진행하면 끊길 가능성이 매우 높음.
    - 실행 시 로컬로 내리고 실행하는 것을 권장함.
    - 모든 평가를 한번에 진행하는 것은 너무 오래 거릴므로, 태스크를 나눠서 진행하도록 하는것을 권장.
``` shell
git clone https://github.com/Beomi/ko-lm-evalutation-harness
cd ko-lm-evaluation-harness
pip isntall -r requirements.txt
./run_all.sh 모델이름 'GPU번호들'

# ex) ./run_all.sh beomi/llama-2-ko-7b '0,1' # 이렇게 하면 GPU 0,1번에 'eomi/llama-2-ko-7b' 모델을 쪼개서 올리고 평가한다.

```


### Ko BEST Dataset
huggingface.co/datasets/skt/kobest_v1/viewer/boolq/train


### Ko BEST Eval Code
!python main.py \
--model hf-causal-experimental \
--model_args pretrained=wonik-hi/new_mistral7B,use_accelerate=true, trust_remote_code=true \
--tasks kobest_hellaswag \
--num_fewshot 0 \
--no_cache \
--batch_size 8 \
--output_path result/all/wonik-hi/new_mistral7B/0_shot_kobest_hellaswag.json

In [ ]:
!git clone https://github.com/Beomi/ko-lm-evaluation-harness
%cd ko-lm-evaluation-harness

In [ ]:
!pip install -r requirements.txt
#./run_all.sh 모델이름 'GPU번호들'

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# 수행 시간 오래 걸림 주의!!!
! ./run_all.sh 'wonik-hi/phi-3-mini-QLoRA' '0'
# ex) ./run_all.sh beomi/llama-2-ko-7b '0,1' # 이렇게 하면 GPU0,1번에 'beomi/llama-2-ko-7b' 모델을 쪼개서 올리고 평가한다.

In [ ]:
# 수행 시간 오래 걸림 주의!!!
# ./run_all.sh model GPU_NO few-shot test
! ./run_all.sh 'oz1115/phi-3-mini-QLoRA' '0' "0" 0.2
# ex) ./run_all.sh beomi/llama-2-ko-7b '0,1' # 이렇게 하면 GPU0,1번에 'beomi/llama-2-ko-7b' 모델을 쪼개서 올리고 평가한다.

In [ ]:
# mixtral eval
!python main.py \
--model hf-causal-experimental \
--model_args pretrained=wonik-hi/new_mistral7B,use_accelerate=true,trust_remote_code=true \
--tasks kobest_hellaswag \
--num_fewshot 0 \
--no_cache \
--batch_size 8 \
--output_path result/all/wonik-hi/new_mistral7B/0_shot_kobest_hellaswag.json

In [ ]:
!python main.py \
--model hf-causal-experimental \
--model_args pretrained=wonik-hi/phi-3-mini-QLoRA,use_accelerate=true,trust_remote_code=true \
--tasks kobest_hellaswag \
--num_fewshot 0 \
--no_cache \
--batch_size 8 \
--output_path result/all/wonik-hi/phi-3-mini-QLoRA2/0_shot_kobest_hellaswag.json

### Results

In [ ]:
import pandas as pd
from glob import glob
import json

# results/korean_origin_bench/20b/00_shot.json
def _get_metric_name(v):
    metrics = ['f1', 'macro_f1', 'acc_norm', 'acc']
    for m in metrics:
        if v.get(m):
            return {
                'metric': m,
                'value': v[m],
            }

def get_df_klue(path, model_name=''):
    data = []
    for i in ['0', '5', '10', '50']:
        shot = f'{path}/{i}_shot.json'
        try:
            data.append(
                {
                    f"{k} ({_get_metric_name(v)['metric']})": _get_metric_name(v)['value']
                    for k, v in json.load(open(shot))['results'].items()
                }
            )
        except FileNotFoundError:
            pass
    df = pd.DataFrame(data, index=[0, 5, 10, 50][:len(data)]).T
    print(df.to_markdown())
    return df

In [ ]:
get_df_klue('results/klue_etc_bench/home/jovyan/beomi/llama2-koen-13b/60b', 'llama2-koen-13b')

In [ ]:
various_models = sorted(glob('results/all/*/*'))
various_models

In [ ]:
for model in various_models:
    print(model)
    get_df_klue(model)
    print()